In [1]:
import os
import pandas
import numpy as np
import torch
import matplotlib.pyplot as plt
import math
import isingmodellight
from isingmodellight import IsingModelLight
from scipy import stats
import time
import hcpdatautils as hcp

In [2]:
int_type = torch.int
float_type = torch.float
device = torch.device('cuda')

In [3]:
file_dir = 'E:\\Ising_model_results_daai'

In [4]:
def depytorch(t:torch.Tensor):
    return t.detach().cpu().numpy()

In [6]:
simple_ts = torch.tensor(data=[[-1,-1,-1,-1,-1],[-1,-1,-1,1,1],[-1,-1,1,-1,-1],[-1,-1,1,1,1],[-1,1,-1,-1,-1],[-1,1,-1,1,1],[-1,1,1,-1,-1],[-1,1,1,1,1],[1,-1,-1,-1,-1],[1,-1,-1,1,1],[1,-1,1,-1,-1],[1,-1,1,1,1],[1,1,-1,-1,-1],[1,1,-1,1,1],[1,1,1,-1,-1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]], dtype=float_type, device=device)
print(simple_ts)
print( simple_ts.size() )

tensor([[-1., -1., -1., -1., -1.],
        [-1., -1., -1.,  1.,  1.],
        [-1., -1.,  1., -1., -1.],
        [-1., -1.,  1.,  1.,  1.],
        [-1.,  1., -1., -1., -1.],
        [-1.,  1., -1.,  1.,  1.],
        [-1.,  1.,  1., -1., -1.],
        [-1.,  1.,  1.,  1.,  1.],
        [ 1., -1., -1., -1., -1.],
        [ 1., -1., -1.,  1.,  1.],
        [ 1., -1.,  1., -1., -1.],
        [ 1., -1.,  1.,  1.,  1.],
        [ 1.,  1., -1., -1., -1.],
        [ 1.,  1., -1.,  1.,  1.],
        [ 1.,  1.,  1., -1., -1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.]], device='cuda:0')
torch.Size([21, 5])


In [16]:
simple_mean = torch.mean(simple_ts, dim=0)
print( 'means', simple_mean )
simple_uncentered_covs = torch.matmul( torch.transpose(simple_ts, dim0=0, dim1=1), simple_ts )/( simple_ts.size(dim=0) )
print('uncentered covariances')
print(simple_uncentered_covs)
simple_centered_covs = simple_uncentered_covs - simple_mean.unsqueeze(dim=0) * simple_mean.unsqueeze(dim=1)
print('centered covarainces')
print(simple_centered_covs)
simple_eigen_vals, simple_eigen_vecs = torch.linalg.eigh(simple_centered_covs)
print('eigenvalues')
print(simple_eigen_vals)
print('eigenvectors')
print(simple_eigen_vecs)

means tensor([0.2381, 0.2381, 0.2381, 0.2381, 0.2381], device='cuda:0')
uncentered covariances
tensor([[1.0000, 0.2381, 0.2381, 0.2381, 0.2381],
        [0.2381, 1.0000, 0.2381, 0.2381, 0.2381],
        [0.2381, 0.2381, 1.0000, 0.2381, 0.2381],
        [0.2381, 0.2381, 0.2381, 1.0000, 1.0000],
        [0.2381, 0.2381, 0.2381, 1.0000, 1.0000]], device='cuda:0')
centered covarainces
tensor([[0.9433, 0.1814, 0.1814, 0.1814, 0.1814],
        [0.1814, 0.9433, 0.1814, 0.1814, 0.1814],
        [0.1814, 0.1814, 0.9433, 0.1814, 0.1814],
        [0.1814, 0.1814, 0.1814, 0.9433, 0.9433],
        [0.1814, 0.1814, 0.1814, 0.9433, 0.9433]], device='cuda:0')
eigenvalues
tensor([2.6774e-09, 7.6190e-01, 7.6190e-01, 1.0656e+00, 2.1271e+00],
       device='cuda:0')
eigenvectors
tensor([[ 0.0000e+00,  0.0000e+00,  8.1650e-01, -5.0775e-01,  2.7481e-01],
        [ 4.1617e-08,  7.0711e-01, -4.0825e-01, -5.0775e-01,  2.7481e-01],
        [-2.5864e-08, -7.0711e-01, -4.0825e-01, -5.0775e-01,  2.7481e-01],
     

In [17]:
full_rank_ts = simple_ts[0:16,0:4]
print( full_rank_ts )
full_rank_mean = torch.mean(full_rank_ts, dim=0)
print( 'means', full_rank_mean )
full_rank_uncentered_covs = torch.matmul( torch.transpose(full_rank_ts, dim0=0, dim1=1), full_rank_ts )/( full_rank_ts.size(dim=0) )
print('uncentered covariances')
print(full_rank_uncentered_covs)
full_rank_centered_covs = full_rank_uncentered_covs - full_rank_mean.unsqueeze(dim=0) * full_rank_mean.unsqueeze(dim=1)
print('centered covarainces')
print(full_rank_centered_covs)
full_rank_eigen_vals, full_rank_eigen_vecs = torch.linalg.eigh(full_rank_centered_covs)
print('eigenvalues')
print(full_rank_eigen_vals)
print('eigenvectors')
print(full_rank_eigen_vecs)

tensor([[-1., -1., -1., -1.],
        [-1., -1., -1.,  1.],
        [-1., -1.,  1., -1.],
        [-1., -1.,  1.,  1.],
        [-1.,  1., -1., -1.],
        [-1.,  1., -1.,  1.],
        [-1.,  1.,  1., -1.],
        [-1.,  1.,  1.,  1.],
        [ 1., -1., -1., -1.],
        [ 1., -1., -1.,  1.],
        [ 1., -1.,  1., -1.],
        [ 1., -1.,  1.,  1.],
        [ 1.,  1., -1., -1.],
        [ 1.,  1., -1.,  1.],
        [ 1.,  1.,  1., -1.],
        [ 1.,  1.,  1.,  1.]], device='cuda:0')
means tensor([0., 0., 0., 0.], device='cuda:0')
uncentered covariances
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]], device='cuda:0')
centered covarainces
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]], device='cuda:0')
eigenvalues
tensor([1., 1., 1., 1.], device='cuda:0')
eigenvectors
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 

In [18]:
unique_vars_ts = simple_ts[:,0:4]
print( unique_vars_ts )
unique_vars_mean = torch.mean(unique_vars_ts, dim=0)
print( 'means', unique_vars_mean )
unique_vars_uncentered_covs = torch.matmul( torch.transpose(unique_vars_ts, dim0=0, dim1=1), unique_vars_ts )/( unique_vars_ts.size(dim=0) )
print('uncentered covariances')
print(full_rank_uncentered_covs)
unique_vars_centered_covs = unique_vars_uncentered_covs - unique_vars_mean.unsqueeze(dim=0) * unique_vars_mean.unsqueeze(dim=1)
print('centered covarainces')
print(unique_vars_centered_covs)
unique_vars_eigen_vals, unique_vars_eigen_vecs = torch.linalg.eigh(unique_vars_centered_covs)
print('eigenvalues')
print(unique_vars_eigen_vals)
print('eigenvectors')
print(unique_vars_eigen_vecs)

tensor([[-1., -1., -1., -1.],
        [-1., -1., -1.,  1.],
        [-1., -1.,  1., -1.],
        [-1., -1.,  1.,  1.],
        [-1.,  1., -1., -1.],
        [-1.,  1., -1.,  1.],
        [-1.,  1.,  1., -1.],
        [-1.,  1.,  1.,  1.],
        [ 1., -1., -1., -1.],
        [ 1., -1., -1.,  1.],
        [ 1., -1.,  1., -1.],
        [ 1., -1.,  1.,  1.],
        [ 1.,  1., -1., -1.],
        [ 1.,  1., -1.,  1.],
        [ 1.,  1.,  1., -1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.]], device='cuda:0')
means tensor([0.2381, 0.2381, 0.2381, 0.2381], device='cuda:0')
uncentered covariances
tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]], device='cuda:0')
centered covarainces
tensor([[0.9433, 0.1814, 0.1814, 0.1814],
        [0.1814, 0.9433, 0.1814, 0.1814],
        [0.1814, 0.1814, 0.9433, 0.181

In [21]:
U, S, V = torch.pca_lowrank(simple_ts, q=5, niter=1000)
print('U')
print(U)
print('S')
print(S)
print('S**2/(m-1)')
print(  S.square()/( simple_ts.size(dim=0) )  )
print('V')
print(V)

U
tensor([[-3.8312e-01,  2.2249e-01,  5.0263e-08, -5.7279e-08,  2.1822e-01],
        [-1.0941e-02,  5.0709e-01, -3.0903e-08, -2.9533e-07,  2.1822e-01],
        [-3.0088e-01,  7.8233e-03,  3.0127e-01,  2.7550e-01,  2.1822e-01],
        [ 7.1295e-02,  2.9242e-01,  3.0127e-01,  2.7550e-01,  2.1822e-01],
        [-3.0088e-01,  7.8231e-03, -3.8923e-01,  1.2316e-01,  2.1822e-01],
        [ 7.1294e-02,  2.9242e-01, -3.8923e-01,  1.2316e-01,  2.1822e-01],
        [-2.1865e-01, -2.0685e-01, -8.7958e-02,  3.9866e-01,  2.1822e-01],
        [ 1.5353e-01,  7.7751e-02, -8.7958e-02,  3.9866e-01,  2.1822e-01],
        [-3.0088e-01,  7.8228e-03,  8.7958e-02, -3.9866e-01,  2.1822e-01],
        [ 7.1295e-02,  2.9242e-01,  8.7958e-02, -3.9866e-01,  2.1822e-01],
        [-2.1865e-01, -2.0685e-01,  3.8923e-01, -1.2316e-01,  2.1822e-01],
        [ 1.5353e-01,  7.7751e-02,  3.8923e-01, -1.2316e-01,  2.1822e-01],
        [-2.1865e-01, -2.0685e-01, -3.0127e-01, -2.7550e-01,  2.1822e-01],
        [ 1.5353e-01,  